# Analiza časovne vrste podatkov Sentinel-2

Analizirali bomo časovno vrsto vegetacijskioh indeksov dobljenih iz podatkov satelita Sentinel-2. Zmogljivosti statističnih analiz časovnih vrst na Sentinel Hubu so opisane:  
https://www.sentinel-hub.com/develop/documentation/api/fis-request

In [ ]:
# Knjižnice
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
# Velikost grafikonov
plt.rcParams['figure.figsize'] = [10, 8]

In [ ]:
# ID ključ mora biti definiran, preberemo ga iz datoteke
sentinelhub_id = 'sentinelhub.id'
f = open(sentinelhub_id,'r')
exec(f.readline())

In [ ]:
if 'INSTANCE_ID' in locals():
    print("Sentinel Hub INSTANCE_ID obstaja")
    print(INSTANCE_ID)

In [ ]:
# Parametri za iskanje
shub_url_base="http://services.sentinel-hub.com/ogc/fis/"
# Ločljivost 10 m
res="&RESOLUTION=10m"
# Gledamo vse posnetke
maxcc="&MAXCC=100"

## Območje

Začnemo z enostavnim poligonom.

In [ ]:
crs="EPSG:4326"
geometry_fn="podatki/kranj_rupa.wkt"

In [ ]:
with open(geometry_fn, 'r') as geo_file:
  geometry = geo_file.read()
time = "2015-01-01/2018-12-31"

## Poizvedba

Preberemo celotno časovno vrsto.

In [ ]:
# URL za poizvedbo
shub_url = shub_url_base + INSTANCE_ID + \
    "?LAYER=NDVI-CLOUDS&STYLE=INDEX&CRS=" +  \
    crs + "&TIME=" + time + "&GEOMETRY=" + geometry + res + maxcc
shub_url

## Prenos podatkov

Časovno vrsto prenesemo v obliki JSON.

In [ ]:
r=requests.get(shub_url)
contents = json.loads(r.content.decode())

In [ ]:
# Ustvarimo pandas dataframe
df = json_normalize(contents['C0'])
df.head()

In [ ]:
# Dodamo še povprečno oblačnost v poligonu
df["clouds"] = json_normalize(contents['C1'])["basicStats.mean"]
df.head()

In [ ]:
# Stolpec date pretvorimo v datum in naredimo indeks
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df = df.set_index("date")
df.head()

In [ ]:
# Podatke shranimo v datoteko
df_fn = "podatki/kranj_rupa.csv"
df.to_csv(df_fn)

In [ ]:
# Izrišemo graf oblačnosti
plt.plot(df["clouds"], linestyle='-', marker='o', color="b", alpha=0.8)
plt.fill_between(df.index, df["clouds"], alpha=0.3)
plt.show()

## Maskiranje podatkov z oblaki

In [ ]:
# Izločimo podatke z več kot 10% oblakov
df = df.mask(df["clouds"] >= 0.1)
df.head()

In [ ]:
df.describe()

In [ ]:
# Ustvarimo podatkovni sloj brez oblakov
df_cl = df.dropna()
df_cl.head()

In [ ]:
# Število malo oblačnih posnetkov po mesecih
df_cl_m = df_cl["clouds"].groupby([(df_cl.index.year),(df_cl.index.month)]).count()
df_cl_m

In [ ]:
df_cl_m

## Graf poteka NDVI

In [ ]:
df_cl["basicStats.mean"].plot(linestyle='-', marker='o')

## Anliza podatkov za leto 2016

Poglejmo kako se je v letu 2016 razvijala vegetacija na različnih površinah. Zanima nas:

* gozd,
* njiva,
* naselje.

In [ ]:
# WKT različnih območij je v CSV datoteki
vzorci_wkt_fn = "./podatki/vzorci_raba_tal_wkt.csv"
vzorci_wkt = pd.read_csv(vzorci_wkt_fn, sep=";")
vzorci_wkt

## Časovne vrste posameznih kategorij

Gornji postopek ponovimo za vse kategorije.

In [ ]:
kat_ndvi = pd.DataFrame()

In [ ]:
for index, row in vzorci_wkt.iterrows():
    print (row["Kategorija"])
    geometry = row["WKT"]
    shub_url = shub_url_base + INSTANCE_ID + \
    "?LAYER=NDVI-CLOUDS&STYLE=INDEX&CRS=" +  \
    crs + "&TIME=" + time + "&GEOMETRY=" + geometry + res + maxcc
    r=requests.get(shub_url)
    contents = json.loads(r.content.decode())
    df = json_normalize(contents['C0'])
    df["clouds"] = json_normalize(contents['C1'])["basicStats.mean"]
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    df = df.set_index("date")
    df = df.mask(df["clouds"] >= 0.1)
    df_cl = df.dropna()
    df_join = df_cl["basicStats.mean"].copy()
    kat_ndvi[row["Kategorija"]] = df_join

In [ ]:
kat_ndvi.head()

In [ ]:
kat_ndvi.plot(marker='o')